In [2]:
import pandas as pd

pred = pd.read_csv("outputs/ltv_predictions.csv")
feat = pd.read_csv("ltv_features_raw.csv")

df = feat.merge(pred, on="customer_id", how="left")
df.head()

,customer_id,frequency,monetary_sum,monetory_avg,last_order_date,first_order_date,distinct_categories,avg_items,recencey_days,tenure_days,orders_30d,orders_90d,orders_180d,predicted_ltv_12m,ltv_segment
0,C0001,2,1650,825.000000,2023-05-11,2023-02-15,2,1.50,600,685,0.0,0.0,0.0,3617.12520,Medium
1,C0002,4,2480,620.000000,2024-03-29,2022-12-28,4,1.75,277,734,0.0,0.0,0.0,2971.21200,Low
2,C0003,4,2790,697.500000,2024-07-19,2023-03-22,4,1.25,165,650,0.0,0.0,1.0,3227.62940,Low
3,C0004,4,4540,1135.000000,2024-11-01,2020-08-12,4,1.50,60,1602,0.0,2.0,2.0,3603.33900,Low
4,C0005,3,1775,591.666667,2024-06-05,2021-06-30,3,1.00,209,1280,0.0,0.0,0.0,830.18604,Low


In [3]:
df['rank'] = df['predicted_ltv_12m'].rank(method='first')
df['decile'] = pd.qcut(df['rank'], 10, labels=False)

decile_summary = df.groupby('decile').agg(
    customers=('customer_id','count'),
    avg_ltv=('predicted_ltv_12m','mean'),
    sum_ltv=('predicted_ltv_12m','sum')
).reset_index()

decile_summary.to_csv("outputs/decile_summary.csv", index=False)
decile_summary

,decile,customers,avg_ltv,sum_ltv
0,0,6,1074.547134,6447.282804
1,1,5,2337.578640,11687.893200
2,2,5,2713.255340,13566.276700
3,3,5,2988.465460,14942.327300
4,4,5,3194.229440,15971.147200
5,5,5,3457.307960,17286.539800
6,6,5,3683.120060,18415.600300
7,7,5,3944.033840,19720.169200
8,8,5,4769.471380,23847.356900
9,9,5,6154.303220,30771.516100


In [4]:
top = df.sort_values("predicted_ltv_12m", ascending=False).head(50)
top.to_csv("outputs/top_customers.csv", index=False)
top.head()

,customer_id,frequency,monetary_sum,monetory_avg,last_order_date,first_order_date,distinct_categories,avg_items,recencey_days,tenure_days,orders_30d,orders_90d,orders_180d,predicted_ltv_12m,ltv_segment,rank,decile
16,C0016,4,3850,962.500000,2024-09-07,2020-04-13,4,1.250000,115,1723,0.0,0.0,1.0,8043.3813,High,51.0,9
9,C0010,4,3640,910.000000,2024-08-19,2021-06-09,4,1.750000,134,1301,0.0,0.0,1.0,6297.1104,High,50.0,9
50,C0050,3,2960,986.666667,2024-11-04,2023-02-26,3,1.333333,57,674,0.0,1.0,1.0,5619.4214,High,49.0,9
26,C0026,4,4050,1012.500000,2024-10-17,2023-12-11,4,1.250000,75,386,0.0,1.0,2.0,5419.2983,High,48.0,9
5,C0006,5,4090,818.000000,2024-04-27,2020-06-30,4,1.400000,248,1645,0.0,0.0,0.0,5392.3047,High,47.0,9
